In [2]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [3]:
import numpy as np

import importlib

import snapshot_obj
import simulation_tracing
import halo_matching

In [59]:
importlib.reload(snapshot_obj)
importlib.reload(simulation_tracing)
importlib.reload(halo_matching)

<module 'halo_matching' from '/home/kasper/Curvaton_Simulations/APOSTLE_analysis/halo_matching.py'>

# Matching all halos between two snapshots

In [14]:
snap1 = snapshot_obj.Snapshot("V1_LR_fix", 127)
gns1 = snap1.get_subhalos("GroupNumber")
sgns1 = snap1.get_subhalos("SubGroupNumber")
masses1 = snap1.get_subhalos('MassType')[:,1]

snap2 = snapshot_obj.Snapshot("V1_LR_fix", 126)
gns2 = snap2.get_subhalos("GroupNumber")
sgns2 = snap2.get_subhalos("SubGroupNumber")
masses2 = snap2.get_subhalos('MassType')[:,1]

In [16]:
%time matches = halo_matching.match_snapshots(snap1,snap2)

CPU times: user 7.17 s, sys: 67.1 ms, total: 7.24 s
Wall time: 7.28 s


In [18]:
print("127 : 126")
for gn1,sgn1,m1,match in zip(gns1,sgns1,masses1,matches):
    if match < 2**32:
        gn2 = gns2[match]
        sgn2 = sgns2[match]
        m2 = masses2[match]
        print("({},{},{:.2e}) : ({},{},{:.2e})".format(gn1,sgn1,m1,gn2,sgn2,m2))
    else:
        print("({},{},{:.2e}) : ({},{})".format(gn1,sgn1,m1,-1,-1))

127 : 126
(1.0,0.0,3.03e+45) : (1.0,0.0,3.00e+45)
(1.0,1.0,5.50e+43) : (1.0,1.0,5.55e+43)
(1.0,2.0,1.40e+43) : (40.0,0.0,1.47e+43)
(1.0,3.0,8.37e+42) : (1.0,2.0,8.57e+42)
(1.0,4.0,7.57e+42) : (1.0,3.0,8.19e+42)
(1.0,5.0,7.19e+42) : (1.0,4.0,8.19e+42)
(1.0,6.0,4.90e+42) : (1.0,5.0,4.90e+42)
(1.0,7.0,2.00e+42) : (1.0,7.0,1.57e+42)
(1.0,8.0,3.57e+42) : (1.0,8.0,3.43e+42)
(1.0,9.0,2.83e+42) : (1.0,6.0,3.10e+42)
(1.0,10.0,3.50e+42) : (1.0,9.0,3.73e+42)
(1.0,11.0,3.47e+42) : (1.0,10.0,3.16e+42)
(1.0,12.0,2.77e+42) : (1.0,11.0,2.73e+42)
(1.0,13.0,2.43e+42) : (1.0,12.0,2.48e+42)
(1.0,14.0,1.85e+42) : (1.0,14.0,2.08e+42)
(1.0,15.0,1.72e+42) : (1.0,13.0,2.11e+42)
(1.0,16.0,1.59e+42) : (1.0,15.0,1.60e+42)
(1.0,17.0,1.17e+42) : (1.0,16.0,1.21e+42)
(1.0,18.0,1.03e+42) : (1.0,17.0,1.11e+42)
(1.0,19.0,1.03e+42) : (1.0,23.0,9.62e+41)
(1.0,20.0,9.62e+41) : (1.0,20.0,9.62e+41)
(1.0,21.0,9.91e+41) : (1.0,19.0,1.06e+42)
(1.0,22.0,9.91e+41) : (1.0,21.0,9.91e+41)
(1.0,23.0,9.18e+41) : (1.0,22.0,9.62e+41)
(1

# Tracing all halos

In [70]:
sim_id = "V1_MR_fix"
snap = snapshot_obj.Snapshot(sim_id, 127)

In [71]:
snap.get_halo_number()

13548

In [72]:
tracer = simulation_tracing.SimulationTracer(snap)

creating file .tracer_V1_MR_fix.hdf5


In [73]:
%time trace = tracer.trace_all(100)

CPU times: user 5.12 ms, sys: 3.88 ms, total: 9 ms
Wall time: 10.3 ms


In [74]:
trace[0][:10]

array([[         0,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                 0,          0,          0,          0,          0,
                 0,          0,          0],
       [       960,        936,        935,        945,       1002,
               985,        968,       1026,       1081,       1107,
              1146,       1133,       1166,       1127,          1,
                 1,          1,          1,          1,          1,
                 1,          1,          1,          1,          1,
                 1,          1,          1],
       [      1502,       1507,       1488,       1521,       1492,
              1562,       1563,       1614,       1575,       1617,
              1649,       1637,       1643,       1647,       1652,
              1627,       

In [27]:
gns = {}; sgns = {}
for sid in range(127,99,-1):
    snap = snapshot_obj.Snapshot(sim_id, sid)
    gns[sid] = snap.get_subhalos("GroupNumber")
    sgns[sid] = snap.get_subhalos("SubGroupNumber")

In [45]:
print(np.arange(gns[127].size)[np.logical_and(gns[127]==2, sgns[127]==0)])
print(gns[127][55], sgns[127][55])
print(gns[126][60], sgns[126][60])

[55]
2.0 0.0
2.0 0.0


In [28]:
tr = tracer.tracer

for i,ht in enumerate(tr):
    out = "127 : ({},{})".format(gns[127][i],sgns[127][i])
    for sid in range(126,99,-1):
        idx = ht[sid]
        if idx == 2**32:
            out += ",  !"
            break
        else:
            out += ",  {} : ({},{})".format(sid, gns[sid][idx],sgns[sid][idx])
    print(out + "\n")

127 : (1.0,0.0),  126 : (1.0,0.0),  125 : (1.0,0.0),  124 : (1.0,0.0),  123 : (1.0,0.0),  122 : (1.0,0.0),  121 : (1.0,0.0),  120 : (1.0,0.0),  119 : (1.0,0.0),  118 : (1.0,0.0),  117 : (1.0,0.0),  116 : (1.0,0.0),  115 : (1.0,0.0),  114 : (1.0,0.0),  113 : (1.0,0.0),  112 : (1.0,0.0),  111 : (1.0,0.0),  110 : (1.0,0.0),  109 : (1.0,0.0),  108 : (1.0,0.0),  107 : (1.0,0.0),  106 : (1.0,0.0),  105 : (1.0,0.0),  104 : (1.0,0.0),  103 : (1.0,0.0),  102 : (1.0,0.0),  101 : (1.0,0.0),  100 : (1.0,0.0)

127 : (1.0,1.0),  126 : (1.0,1.0),  125 : (1.0,1.0),  124 : (1.0,1.0),  123 : (1.0,1.0),  122 : (1.0,1.0),  121 : (1.0,1.0),  120 : (1.0,1.0),  119 : (1.0,1.0),  118 : (1.0,1.0),  117 : (1.0,1.0),  116 : (1.0,1.0),  115 : (1.0,1.0),  114 : (1.0,1.0),  113 : (7.0,0.0),  112 : (7.0,0.0),  111 : (7.0,0.0),  110 : (7.0,0.0),  109 : (7.0,0.0),  108 : (8.0,0.0),  107 : (8.0,0.0),  106 : (8.0,0.0),  105 : (8.0,0.0),  104 : (8.0,0.0),  103 : (8.0,0.0),  102 : (7.0,0.0),  101 : (7.0,0.0),  100 : (7.0,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

